Зчитуємо файл

In [2]:
import os
def read_file(default="./test_text_small.txt"):
    with open(default, mode = 'r', encoding = 'utf-8') as infile:
        unfiltered = infile.read().lower() #reading file, bringing to lowertext
        unfiltered = ''.join(unfiltered.split())

    #Фільтрація
    allowed_no_whitespace = 'абвгдежзийклмнопрстуфхцчшщъыьэюя'
    text = ''
    for char in unfiltered:
        if char in allowed_no_whitespace:
            text+=char
        if char =='ё':
            text+='е'  
    return text

In [3]:
filtered_text = read_file()

Створимо зразки для аналізу

In [4]:
#Алфавіт в цифру
def to_numbers(text):
    alphabet = 'абвгдежзийклмнопрстуфхцчшщъыьэюя'
    alphabet = dict(zip(alphabet,range(len(alphabet))))

    return [alphabet.get(x) for x in text]
#навпаки
def to_chars(text):
    alphabet = 'абвгдежзийклмнопрстуфхцчшщъыьэюя'
    alphabet = dict(zip(alphabet,range(len(alphabet))))
    alphabet = dict((v,k) for k,v in alphabet.items())
    return [alphabet.get(x) for x in text]


In [5]:
def encode_all(text, key):
    key = to_numbers(key)
    text = to_numbers (text)
    for x in range(len(text)):
        text[x] =  (int(text[x]) + int(key[x%len(key)])) % 32
    return to_chars(text)

In [6]:
keys = 'ок юнг карл добро Абитуриент абракадабра аббревиатура Автобиография Абстракционизм Безнравственный Абстрагироваться Безукоризненность Вероотступничество Достопримечательный Лжесвидетельствовать'.lower()
keys = keys.split(" ")
for key in keys:
	text = ''.join(encode_all(filtered_text,key))
	with open(f"./samples/{key}.txt", 'w', encoding= 'utf-8') as outfile:
		outfile.write(text)

Створимо н-грами

In [7]:
from collections import Counter

def ngram_ctr(letters):
    keys = Counter(letters).keys() #stores n-gram to keys
    values = [x for x in Counter(letters).values()] #counts corresponding frequencies
    #creating a new dict
    retval =  dict(zip(keys, values))
    return dict(sorted(retval.items(), key=lambda item: item[1], reverse=True)) #sorting dict from highest to lowest

def ngrams(letters):
    monogr = [letters[iter:iter+1] for iter in range(0,len(letters),1)] #stores each symbol to list, like [a, b, c, ...]
    return monogr
    
def combined(text):
    return ngram_ctr(ngrams(text))

monogr_no_whitespaces = ngrams(filtered_text)

print("Ngrams::")
print(monogr_no_whitespaces[:10])

print("Counted ngrams::")
monogr_no_whitespaces = ngram_ctr(monogr_no_whitespaces)

print(list(monogr_no_whitespaces.items())[:3])

Ngrams::
['с', 'п', 'а', 'с', 'и', 'б', 'о', 'ч', 'т', 'о']
Counted ngrams::
[('о', 29301), ('а', 24239), ('н', 18678)]


In [8]:
def Index(monograms):
	counts = monograms.values()
	length = sum(counts)
	sums = []
	for item in counts:
		sums.append((item * item - 1) / (length * (length - 1)))
	return sum(sums)

Індекс відповідності незашифрованого тексту

In [9]:
print(f"Індекс відповідності тексту {Index(monogr_no_whitespaces)}")

Індекс відповідності тексту 0.051725509617763156


In [10]:
import os
samples = os.listdir("./samples")
samples.sort(key=len)
samples_len = [len(item.split('.')[0]) for item in samples]
for file in range(len(samples)):
    text = read_file("./samples/"+samples[file])
    print(f"{samples[file]}:: довжина == {samples_len[file]}:: {Index(combined(text))}")
    

ок.txt:: довжина == 2:: 0.04145869979506963
юнг.txt:: довжина == 3:: 0.036634244014512776
карл.txt:: довжина == 4:: 0.035738788564177615
добро.txt:: довжина == 5:: 0.036171210361810716
абитуриент.txt:: довжина == 10:: 0.03388476873148918
абракадабра.txt:: довжина == 11:: 0.03633602960565197
аббревиатура.txt:: довжина == 12:: 0.034237802416873894
автобиография.txt:: довжина == 13:: 0.03331872483072929
абстракционизм.txt:: довжина == 14:: 0.03268068386986754
безнравственный.txt:: довжина == 15:: 0.0330052143078738
абстрагироваться.txt:: довжина == 16:: 0.033304581730989644
безукоризненность.txt:: довжина == 17:: 0.03336284484440337
вероотступничество.txt:: довжина == 18:: 0.03366149534243784
достопримечательный.txt:: довжина == 19:: 0.03262808140553253
лжесвидетельствовать.txt:: довжина == 20:: 0.03303088640013991


Розшифровка

In [11]:
#decode
def decode_one(character, keychar):
    return (character-keychar)%32
    ...
def decode_all(text, key):
    text = to_numbers(text)
    key = to_numbers(key)
    for x in range(len(text)):
        text[x] = decode_one(text[x],key[x%len(key)])
    return to_chars(text)   


In [12]:
# decoding procedure
data = read_file('./7.txt')
results = []
for length in range(2,31):
    substing_index = []
    for substring in range(length):
        substing_index.append(Index(combined(data[substring::length])))
    results.append(sum(substing_index)/len(substing_index))


for i in range(len(results)):
    posible = []
    if results[i] >= 0.05:
        print(i+2, results[i])
        posible.append(i+2)
        print(f'posibble key length is {posible}')

15 0.058245951219597504
posibble key length is [15]
30 0.060111105793404
posibble key length is [30]


In [13]:
#вводимо можливу довжину ключа
key_length = 15
#найчастіші букви
common = "оаеин"

temp = []
for length in range(2,key_length+1):
    substings = ''
    for substring in range(length):
        x = combined(data[substring::length])
        substings+=(list(x.keys())[0])

print(substings)

for i in common:
    print(''.join(decode_all(substings,i)))



оюбтохуроюгцъос
арудазевархимаг
оюбтохуроюгцъос
йщьнйролйщюсхйм
жцщкжнлижцыотжй
бсфебижгбсцйнбд


можливий ключ - арудазевархимаг

In [14]:
key = 'арудазевархимаг'
print(data[:len(key)])
decoded = ''.join(decode_all(data,key))
print(decoded[:len(key)]) 

пабьлхэбтэхмвах
прошлошятнадцат


можливий ключ - арудазевархимаг
результат -     прошлошятнадцат

ШТ      пабьлх э бтэхмвах
ключ    арудаз е вархимаг
ВТ      прошло ш ятнадцат

буква ш не підходить, спробуємо підібрати

In [15]:
SHT = to_numbers('э')[0]
VT = to_numbers('п')[0]
for key in 'абвгдежзийклмнопрстуфхцчшщъыьэюя':
    if decode_one(SHT,to_numbers(key)[0]) == VT:
        print(key, to_numbers(key)) 

о [14]


отже спробуємо ключ арудазовархимаг

In [16]:
key = 'арудазовархимаг'
print(data[:len(key)])
decoded = ''.join(decode_all(data,key))
print(decoded[:len(key)])  

пабьлхэбтэхмвах
прошлопятнадцат


In [17]:
text = ''.join(decoded)
with open(f"./{key}.txt", 'w', encoding= 'utf-8') as outfile:
	outfile.write(text)